Method(s)

In [79]:
import numpy as np
import random
import string

In [81]:
def readData(file_names):
    #read data
    corpus = ""
    file_names = 'toy_data.txt'
    f = open(file_names, 'r')
    corpus = f.read()
    #encode text string
    corpus = corpus.replace('\n',' ')
    corpus = corpus.replace('\t',' ')
    corpus = corpus.replace('"', ' ')
    corpus = corpus.replace('"', ' ')
    for spaced in ['.','-',',','!','?','(','—',')']:
        corpus = corpus.replace(spaced, ' {0} '.format(spaced))
    corpus = corpus.replace('\n',' ')
    corpus = corpus.replace('\t',' ')
    corpus = corpus.replace('"','')
    corpus = corpus.replace('"','')
    for spaced in ['.','-',',','!','?','(','—',')',':']:
        corpus = corpus.replace(spaced,' ')
    dic_corpus = corpus.split()
    #if use dictionary, it'll sort by itself
    distinct_corpus = sorted(set(dic_corpus))
    return dic_corpus,distinct_corpus

def cleanup_data(sentence):
    return sentence.translate(str.maketrans('','', string.punctuation))

def create_dict(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = []
    dictionary[key].append(value)

#create probability dictionary 
def probability_dict(list_data):
    probability_dict = {}
    given_list_length = len(list_data)
    for item in list_data:
        probability_dict[item] = probability_dict.get(item, 0) + 1
    for key, value in probability_dict.items():
        probability_dict[key] = value / given_list_length
    return probability_dict

def Mat2Jason(matrix):
    JMatrix = {}
    for i,row in zip(list(range(np.size(matrix,axis=0))),matrix):
        temp = {}
        for j,val in zip(list(range(np.size(row,axis=0))),row):
            temp[j] = val
        JMatrix[i]=temp
    return JMatrix
  
def TranMat(encodedCorpus):
    nStates = len(encodedCorpus)
    tranMat = np.zeros(shape=(nStates,nStates))
    for i in range(0,nStates):
        fndEl = list(np.where(encodedCorpus==i)[0])
        for el in fndEl:
              if(el +1 != len(encodedCorpus)):
                tranMat[i,encodedCorpus[el+1]] += 1/len(fndEl)
    
    ''' Laplace Smoothing to normalize the matrix '''
#     tranMat[tranMat!=0] = TranMat[TranMat!=0] + (1/nStates)
#     TranMat[TranMat==0] = 1/nStates
    return tranMat

def emisMat():
    pass
    
def ForwardAlg(observations, initialprob, trans, emis, numstates, observation_indices):
    alpha = np.zeros((numstates, len(observations)))
      # initialization
    obs_index = observation_indices[observations[0]]
    for s in range(numstates):
          forwardmatrix[s,0] = initialprob[s]*emis[s,obs_index]

      # recursion
    for t in range(1, len(observations)):
        obs_index = observation_indices[observations[t]]
        for s in range(numstates):
            forwardmatrix[s,t] = emis[s,obs_index]*sum([forwardmatrix[s2,t-1]*trans[s2,s] for s2 in range(numstates)])
    return forwardmatrix


def BackwardAlg(observations, trans, emis, numstates, observation_indices):
    backwardmatrix = numpy.zeros((numstates, len(observations)))
      # initialization
    for s in range(numstates):
        backwardmatrix[ s, len(observations) - 1 ] = 1.0

      # recursion
    for t in range(len(observations) - 2, -1, -1):
        obs_index = observation_indices[ observations[t+1]]
        for s in range(numstates):
            backwardmatrix[s,t] = sum([trans[s,s2]*emis[s2,obs_index]*backwardmatrix[s2,t+1] for s2 in range(numstates)])

    return backwardmatrix


#List to hold the initial states and transition states
initial_word = {}
second_word = {}
transitions = {}
for line in open('alllines.txt'):
    tokens = cleanup_data(line.rstrip().lower()).split()
    tokens_length = len(tokens)
    for i in range(tokens_length):
        token = tokens[i]
        if i == 0:
            initial_word[token] = initial_word.get(token, 0) + 1
        else:
            prev_token = tokens[i - 1]
            if i == tokens_length - 1:
                create_dict(transitions, (prev_token, token), 'END')
            if i == 1:
                create_dict(second_word, prev_token, token)
            else:
                prev_prev_token = tokens[i - 2]
                create_dict(transitions, (prev_prev_token, prev_token), token)

# Normalize the distributions
initial_word_total = sum(initial_word.values())
for key, value in initial_word.items():
    initial_word[key] = value / initial_word_total

for prev_word, next_word_list in second_word.items():
    second_word[prev_word] = probability_dict(next_word_list)

for word_pair, next_word_list in transitions.items():
    transitions[word_pair] = probability_dict(next_word_list)
# Init_prob = probability_dict(dic_corpus)
# #Init_prob
# observations = distinct_corpus
# #create dict for observation w/ index
# observation_indices = dict(zip(observations, range(len(observations))))
# #trans = TranMat(dic_corpus)
# numstates = len(observations)
# tranMat = TranMat(observations)
# emisMat = tranMat.transpose(1,0)
# obs_index = observation_indices[observations[1]]
# tranMat   

try EM

In [70]:
# expectation step:
# re-estimate xi_t(i, j) and gamma_t(j)
# returns two things:
# - gamma is a (N, T) numpy matrix
# - xi is a list of T numpy matrices of size (N, N)
def expectation(observations, trans, emis, numstates, observation_indices, forward, backward):
    # denominator: P(O | HMM)
    p_o_given_hmm = sum([forward[s_i, len(observations) -1] for s_i in range(numstates) ])
   
    # computing xi
    xi = [ ]
    for t in range(len(observations) - 1):
        obs_index = observation_indices[observations[t+1]]
       
        xi_t = np.zeros((numstates, numstates))
       
        for s_i in range(numstates):
            for s_j in range(numstates):
                xi_t[ s_i, s_j] = (forward[s_i, t] * trans[s_i, s_j] * emis[s_j, obs_index] * backward[s_j, t+1]) / p_o_given_hmm
        xi.append(xi_t)

    # computing gamma
    gamma = np.zeros((numstates + 2, len(observations)))
    for t in range(len(observations) - 1):
        for s_i in range(numstates):
            gamma[s_i, t] = sum([ xi[t][s_i, s_j] for s_j in range(numstates) ])

    for s_j in range(numstates):
        gamma[s_j, len(observations) - 1] = sum( [ xi[t][s_i, s_j] for s_i in range(numstates) ] )
           
    return (gamma, xi)

###
# maximization step:
# re-estimate trans, emis based on gamma, xi
# returns:
# - initialprob
# - trans
# - emis
def maximization(observations, gamma, xi, numstates, observation_indices, vocabsize):
    # re-estimate initial probabilities
    initialprob = np.array([gamma[s_i, 0] for s_i in range(numstates)])
   
    # re-estimate emission probabilities
    emis = np.zeros((numstates, vocabsize))

    for s in range(numstates):
        denominator = sum( [gamma[s, t] for t in range(len(observations))])
        for vocab_item, obs_index in observation_indices.items():
            emis[s, obs_index] = sum( [gamma[s, t] for t in range(len(observations)) if observations[t] == vocab_item] )/denominator

    # re-estimate transition probabilities
    trans = np.zeros((numstates, numstates))

    for s_i in range(numstates):
        denominator = sum( [gamma[s_i, t] for t in range(len(observations) - 1) ])
       
        for s_j in range(numstates):
            trans[s_i, s_j] = sum( [ xi[t][s_i, s_j] for t in range(len(observations) - 1) ] )/denominator


    return (initialprob, trans, emis)

# forward function: returns numpy matrix of size (N, T)
def forwardprobs(observations, initialprob, trans, emis, numstates, observation_indices):
    forwardmatrix = np.zeros((numstates, len(observations)))

    # initialization
    obs_index = observation_indices[ observations[0]]
    for s in range(numstates):
        forwardmatrix[ s, 0 ] = initialprob[s] * emis[ s, obs_index]

    # recursion step
    for t in range(1, len(observations)):
        obs_index = observation_indices[ observations[t]]
        for s in range(numstates):
            forwardmatrix[s, t] = emis[s, obs_index] * sum([forwardmatrix[s2, t-1] * trans[s2, s] \
                                       for s2 in range(numstates)])
    return forwardmatrix

# beta_t(j) = P(o_{t+1}, ..., o_T | qt = j, HMM)
# backward function: returns numpy matrix of size (N, T)
def backwardprobs(observations, trans, emis, numstates, observation_indices):
    backwardmatrix = np.zeros((numstates, len(observations)))

    # initialization
    for s in range(numstates):
        backwardmatrix[ s, len(observations) - 1 ] = 1.0

    # recursion
    for t in range(len(observations) - 2, -1, -1):
        obs_index = observation_indices[ observations[t+1]]
        for s in range(numstates):
            backwardmatrix[s, t] = sum([ trans[s, s2] * emis[s2, obs_index] * backwardmatrix[s2, t+1] \
                                         for s2 in range(numstates) ])

    return backwardmatrix
# HMM initialization
# initialize initial probs
obs_indices = dict(zip(observations, range(len(observations))))
nStates = len(encodedCorpus)
vocabsize = len(encodedCorpus)
unnormalized = np.random.rand(nStates)
initialprob = unnormalized / sum(unnormalized)
numiter = 5
# initialize emission probs
emis = np.zeros((nStates, vocabsize))
for s in range(nStates):
    unnormalized = np.random.rand(vocabsize)
    emis[s] = unnormalized / sum(unnormalized)

# initialize transition probs
trans = np.zeros((nStates, nStates))
for s in range(nStates):
    unnormalized = np.random.rand(nStates)
    trans[s] = unnormalized / sum(unnormalized)

print("OBSERVATIONS:")
print(observations)
print("\n")

print("Random initialization:")
print("INITIALPROB")
print(initialprob)
print("\n")

print("EMIS")
print(emis)
print("\n")

print("TRANS")
print(trans)
print("\n")
for iteration in range(numiter):

    forward = forwardprobs(observations, initialprob, trans, emis, nStates, obs_indices)
    backward = backwardprobs(observations, trans, emis, nStates, obs_indices)

    gamma, xi = expectation(observations, trans, emis, nStates, obs_indices, forward, backward)

    initialprob, trans, emis = maximization(observations, gamma, xi, nStates, obs_indices, vocabsize)

print("Re-computed:")
print("INITIALPROB")
print(initialprob)
print("\n")

print("EMIS")
print(emis)
print("\n")

print("TRANS")
print(trans)
print("\n")

print("GAMMA(1)")
print(gamma[0])
print("\n")

print("GAMMA(2)")
print(gamma[1])
print("\n")

OBSERVATIONS:
[3, 2]


Random initialization:
INITIALPROB
[0.037526   0.00944889 0.03466626 0.02945669 0.01022185 0.02625524
 0.01549539 0.02834354 0.01309296 0.00806054 0.01947283 0.02131691
 0.03758229 0.01921765 0.02403569 0.02831827 0.02998779 0.01355308
 0.0056531  0.03203963 0.00170184 0.00663279 0.02973708 0.02456977
 0.0074998  0.03775232 0.026192   0.03161647 0.03632377 0.01545876
 0.02861992 0.01279833 0.02792261 0.01624828 0.03826539 0.01493279
 0.00141517 0.03840007 0.00735803 0.02096796 0.00752855 0.00058176
 0.03436286 0.02774676 0.02790723 0.03371512]


EMIS
[[0.02604598 0.03655737 0.01536641 ... 0.00367525 0.03430004 0.02123023]
 [0.04260959 0.02503871 0.02586685 ... 0.0042876  0.00436766 0.00777747]
 [0.01117842 0.0246133  0.02436358 ... 0.02106072 0.01078962 0.00467217]
 ...
 [0.02424153 0.00149627 0.00901634 ... 0.03954017 0.00480719 0.01583308]
 [0.03971759 0.01090224 0.00508391 ... 0.0219325  0.04271273 0.01073496]
 [0.0218303  0.0356976  0.01784871 ... 0.03888104 

In [87]:
def ViterbiAlg(trans, emis, observations):
    num_obs = len(observations)
    num_states = tranMat.shape[0]
    log_probs = np.zeros(num_states)
    paths = np.zeros( (num_states, num_obs+1 ))
    paths[:, 0] = np.arange(num_states)    
    for obs_ind, obs_val in enumerate(observations):
        for state_ind in range(num_states):
            val = 0
            if obs_val< np.size(EmissMat,1):
                val = np.log(EmissMat[state_ind, obs_val])
            temp_probs = log_probs + \
                          val + \
                         np.log(tranMat[:, state_ind])
            best_temp_ind = np.argmax(temp_probs)
            paths[state_ind,:] = paths[best_temp_ind,:]
            paths[state_ind,(obs_ind+1)] = state_ind
            log_probs[state_ind] = temp_probs[best_temp_ind]
    best_path_ind = np.argmax(log_probs)
    
    return (paths[best_path_ind], log_probs[best_path_ind])

def text_prediction(text):
        text = cleanup_data(text.lower()).split()
        # Initial word
        word0 = text[0]
        # Second word
        if len(text) == 1:
            word1 = sample_word(second_word[word0])
            text.append(word1)
        else:
            word1 = text[1]
        # Subsequent words untill END
        while True:
            word2 = max(transitions[(word0, word1)], key=transitions[(word0, word1)].get)
            if word2 == 'END':
                break
            text.append(word2)
            word0 = word1
            word1 = word2
        print(' '.join(text))

1.Generating new text from the text corpus

In [68]:
def sample(dictionary):
    p0 = np.random.random()
    cumulative = 0
    for key, value in dictionary.items():
        cumulative += value
        if p0 < cumulative:
            return key
    assert(False)

In [82]:
# Function to generate sample text
def generate_text(number_of_sentences):
    for i in range(number_of_sentences):
        sentence = []
        # Initial word
        word0 = sample_word(initial_word)
        sentence.append(word0)
        # Second word
        word1 = sample_word(second_word[word0])
        sentence.append(word1)
        # Subsequent words untill END
        while True:
            word2 = sample_word(transitions[(word0, word1)])
            if word2 == 'END':
                break
            sentence.append(word2)
            word0 = word1
            word1 = word2
        print(' '.join(sentence))

In [83]:
dic_corpus,distinct_corpus = readData('alllines.txt')
generate_text(5)

he shall not save him sir within himself
my thoughts and troubles
come sir i will confess what i speak too
and their army
as priam was for not being at your hand and smiles on the commonwealth and made her neighbours believe she


2.Perform text prediction on sequantial words

In [90]:
text_prediction("my marvel")

my marvel and my good lord
